In [1]:
from quilt.data.akarve import fivethirtyeight as fte

In [9]:
air_df = fte.airline_safety.airline_safety()

In [3]:
import t4

/Users/karve/anaconda3/envs/dev/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [6]:
t4.put(air_df, "alpha-quilt-storage/s3select/airline_safety.parquet")

In [10]:
import boto3

In [11]:
s3 = boto3.client("s3")

In [12]:
s3

In [51]:
resp = s3.select_object_content(
    Bucket='alpha-quilt-storage',
    Key='s3select/airline_safety.parquet',
    Expression="SELECT * FROM S3Object s",
    ExpressionType="SQL",
    InputSerialization={
        "Parquet": {}
    },
    OutputSerialization={
        'CSV': {
            'QuoteFields': 'ASNEEDED',
            'RecordDelimiter': '\n',
            'FieldDelimiter': ',',
            'QuoteCharacter': '"'
        }
    },
    RequestProgress={
        'Enabled': True
    }
)

In [52]:
event_stream = resp['Payload']

In [53]:
for event in event_stream:
    # If we received a records event, write the data to a file
    if 'Records' in event:
        data = event['Records']['Payload']
        f.write(data)
    # If we received a progress event, print the details
    elif 'Progress' in event:
        print(event['Progress']['Details'])
    # End event indicates that the request finished successfully
    elif 'End' in event:
        print('Result is complete')
        end_event_received = True

Result is complete


EventStreamError: An error occurred (InternalError) when calling the SelectObjectContent operation: We encountered an internal error. Please try again.